In [1]:
import os
from pyspark.sql import SparkSession

#spark = SparkSession.builder.master("local").appName("mllib_classifier").getOrCreate()
spark = SparkSession.builder.appName("StockPricePrediction").getOrCreate()

# Load training data
filename = "KO.csv"
df = spark.read.csv(filename,  inferSchema=True, header = True)
#data.show(2)

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/02 22:05:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data.describe

<bound method DataFrame.describe of DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Adj Close: double, Volume: int]>

In [2]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Convert the date column to a timestamp
df = df.withColumn("Date", F.to_date("Date"))

# Define a window specification
window_spec = Window().orderBy("Date")

# Calculate the rate of return using the lag function
df = df.withColumn("prev_Close", F.lag("Close").over(window_spec))
df = df.withColumn("prev_Open", F.lag("Open").over(window_spec))
df = df.withColumn("prev_High", F.lag("High").over(window_spec))
df = df.withColumn("prev_Low", F.lag("Low").over(window_spec))
df = df.withColumn("prev_Volume", F.lag("Volume").over(window_spec))

# Calculate the rate of return
df = df.withColumn("Rate_of_Return", ((df["Close"] - df["prev_Close"]) / df["prev_Close"]) * 100)

# Drop the columns used for lagging
#df = df.drop("prev_close") #, "prev_close", "prev_high", "prev_low", "prev_volume")

# Show the result
df.show()


23/12/02 22:06:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:06:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:06:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:06:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:06:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+----------+--------+--------+--------+--------+---------+-------+----------+---------+---------+--------+-----------+-------------------+
|      Date|    Open|    High|     L

In [6]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Convert the date column to a timestamp
df = df.withColumn("Date", F.to_date("Date"))

# Define a window specification
window_spec = Window().orderBy("Date")

# Calculate the rate of return using the closing prices
df = df.withColumn("Rate_of_Return", (df["Close"] - F.lag(df["Close"]).over(window_spec)) / F.lag(df["Close"]).over(window_spec))

# Calculate the rate of return using the closing prices
#stock_data = stock_data.withColumn("Rate_of_Return", (col("close") - lag("close").over(window_spec)) / lag("close").over(window_spec))


# Show the DataFrame with the new "Rate_of_Return" column
df.show()

23/12/02 21:44:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 21:44:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 21:44:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 21:44:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 21:44:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+----------+--------+--------+--------+--------+---------+-------+--------------------+
|      Date|    Open|    High|     Low|   Close|Adj Close| Volume|      Rate_of_Return|

In [3]:
# Drop rows with null values in the "Rate_of_Return" column
df = df.dropna(subset=["Rate_of_Return"])

# Show the result
df.show()

23/12/02 22:06:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:06:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:06:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:06:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:06:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+----------+--------+--------+--------+--------+---------+-------+----------+---------+---------+--------+-----------+-------------------+
|      Date|    Open|    High|     L

In [15]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml import Pipeline
#from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create a Spark session
#spark = SparkSession.builder.appName("StockPricePrediction").getOrCreate()

# Assuming your data is in a DataFrame called "data"
# Replace "target_column" with the actual column name of your target variable
target_column = "Close"
#features_columns = ["Open", "High", "Low", "Close", "Volume"]
features_columns = ["Open", "prev_High", "prev_Low", "prev_Close", "prev_Volume"]
#"High", "Low", "Volume", "prev_Open",

# Assemble the features into a vector
assembler = VectorAssembler(inputCols=features_columns, outputCol="features")
data_assembled = assembler.transform(df)

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(data_assembled)
scaledData = scalerModel.transform(data_assembled)

# Split the data into training and testing sets
(train_data, test_data) = scaledData.randomSplit([0.8, 0.2], seed=333)

# Initialize the logistic regression model
lr = GeneralizedLinearRegression(labelCol=target_column, featuresCol="scaledFeatures",
                      family="gaussian", link="identity",
                      maxIter=10,
                      regParam=0.3) 

# Create a pipeline with the assembler and the logistic regression model
#pipeline = Pipeline(stages=[lr])

# Train the model
model = lr.fit(train_data)

# Print the weights and intercept for linear regression
print("Weights: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

summary = model.summary
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
print("Deviance Residuals: ")
summary.residuals().show()

# Make predictions on the test set
#predictions = model.transform(test_data)

# Evaluate the model using a binary classification evaluator
#evaluator = BinaryClassificationEvaluator(labelCol=target_column)
#accuracy = evaluator.evaluate(predictions)

#print(f"Accuracy: {accuracy}")

# Optionally, you can print the coefficients of the logistic regression model
#print(f"Coefficients: {model.stages[-1].coefficients}")


23/12/02 22:25:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:25:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:25:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:25:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:25:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:25:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 2

In [18]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as F

# compute predictions. this will append column "prediction" to dataframe
lrPred = model.transform(test_data)
#lrPred = lrPred.withColumn("prediction", F.abs(lrPred["prediction"]))
lrPred = lrPred.withColumn("prediction", (lrPred["prediction"]))
lrPred = lrPred.withColumn("Difference", (lrPred["prediction"] - lrPred["Close"]))
lrPred.select("Close","prediction","Difference").show(50)

ev = RegressionEvaluator(predictionCol="prediction", labelCol="Close")

print("METRICS")
print("Mean Squared Error:", ev.evaluate(lrPred, {ev.metricName: "mse"}))
print("R Squared:", ev.evaluate(lrPred, {ev.metricName:'r2'}))

23/12/02 22:28:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:28:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:28:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:28:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/02 22:28:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+--------+-------------------+--------------------+
|   Close|         prediction|          Difference|
+--------+-------------------+--------------------+
| 0.25651|0.3036743

In [ ]:
from pyspark.ml.classification import LogisticRegression

# instantiate the model
lr = LogisticRegression(labelCol='high_price',
                        featuresCol='scaledFeatures',
                        maxIter=10, 
                        regParam=0.3, 
                        elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(scaledData)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# compute predictions. this will append column "prediction" to dataframe
lrPred = lrModel.transform(scaledData)
lrPred.select('probability','prediction').show(5,truncate=False)

# set up evaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction",
                                          labelCol="high_price",
                                          metricName="areaUnderPR")

# pass to evaluator the DF with predictions, labels
aupr = evaluator.evaluate(lrPred)

print("Area under PR Curve:", aupr)